In [ ]:
!unzip drive-download-20241220T223313Z-001.zip -d pdfs


In [ ]:
import os

print(os.name)  # 'posix' for Linux/Mac, 'nt' for Windows


In [ ]:
!nvcc --version

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install pymupdf            # For 'fitz' (PyMuPDF)
!pip install pillow             # For 'PIL' (Image)
!pip install langchain          # For 'langchain.text_splitter'
!pip install transformers       # For 'transformers.AutoTokenizer' and 'transformers.AutoModelForCausalLM'
!pip install torch              # For 'torch'         # For 'faiss' (CPU version; use 'faiss-gpu' for GPU sup!port)
!pip install numpy   

In [ ]:
!pip install faiss-gpu

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import faiss
import numpy as np
import os
import glob
import io

# Путь к папке с PDF файлами
pdf_folder_path = "pdfs"


In [ ]:
# Функция для обработки PDF файлов
def process_pdfs_from_folder(pdf_folder_path):
    all_text, all_images = [], []
    pdf_files = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

    for pdf_path in pdf_files:
        print(f"Обрабатываем файл: {pdf_path}")
        text, images = [], []

        # Открытие и извлечение данных из PDF
        with fitz.open(pdf_path) as pdf:
            for page in pdf:
                text.append(page.get_text())  # Извлекаем текст

                # Извлекаем и сохраняем изображения
                for img_index, img in enumerate(page.get_images(full=True)):
                    xref = img[0]
                    base_image = pdf.extract_image(xref)
                    image_bytes = base_image["image"]
                    image = Image.open(io.BytesIO(image_bytes))
                    image_bytes_io = io.BytesIO()
                    image.save(image_bytes_io, format=base_image["ext"].upper())
                    images.append(image_bytes_io.getvalue())

        all_text.append(" ".join(text))
        all_images.extend(images)

    return all_text, all_images, pdf_files


In [ ]:
# Разбиение текста на чанки
def split_text_into_chunks(text):
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
    return splitter.split_text(text)


In [ ]:
!pip install --upgrade transformers

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [ ]:
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
import faiss
import re

In [ ]:

model_st = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Чистка текста
def clean_text(text):
    text = re.sub(r"[^а-яА-Яa-zA-Z0-9\s.,!?]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_chunks(chunks):
    cleaned_chunks = [clean_text(chunk) for chunk in chunks]
    filtered_chunks = [chunk for chunk in cleaned_chunks if len(chunk.split()) > 5]
    return filtered_chunks

# Сохранение и загрузка Faiss-индекса
def save_embeddings_to_faiss(embeddings):
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    faiss.write_index(index, "faiss_index.index")
    return index

def load_faiss_index(index_path):
    return faiss.read_index(index_path)

def get_embeddings_st(texts):
    return model_st.encode(texts, show_progress_bar=True)

def rank_with_bm25(chunks, query):
    tokenized_chunks = [chunk.split() for chunk in chunks]
    bm25 = BM25Okapi(tokenized_chunks)
    query_tokens = query.split()
    scores = bm25.get_scores(query_tokens)
    ranked_chunks = sorted(
        zip(chunks, scores), key=lambda x: x[1], reverse=True
    )
    return ranked_chunks

def search_similar_chunks_with_images(query, all_chunks, chunk_image_map, index, embeddings, k=5):
    """
    Searches for similar text chunks using Faiss, ranks them with BM25, and retrieves corresponding images.
    """
    # Perform Faiss search
    query_embedding = get_embeddings_st([query])[0]
    D, I = index.search(np.array([query_embedding]), k=10)  # Retrieve top-10 results from Faiss

    # Get text chunks from Faiss results
    similar_chunks = [all_chunks[idx] for idx in I[0]]

    # Rank the chunks using BM25
    ranked_chunks = rank_with_bm25(similar_chunks, query)

    # Retrieve top-k chunks and their corresponding images
    similar_chunks_with_images = []
    for chunk, _ in ranked_chunks[:k]:
        idx = all_chunks.index(chunk)  # Find the original index of the chunk
        image = chunk_image_map.get(idx)  # Get the associated image
        similar_chunks_with_images.append((chunk, image))

    return similar_chunks_with_images


In [ ]:
def save_images(images):
    os.makedirs("images", exist_ok=True)
    image_paths = []
    for idx, image in enumerate(images):
        image_path = os.path.join("images", f"image_{idx}.png")
        with open(image_path, "wb") as f:
            f.write(image)
        image_paths.append(image_path)
    return image_paths

In [ ]:
def process_pdfs(pdf_folder_path):
    all_text, all_images, pdf_files = process_pdfs_from_folder(pdf_folder_path)
    print('1done')
    # Разбиение текста на чанки
    all_chunks = []
    for text in all_text:
        all_chunks.extend(split_text_into_chunks(text))
    print('2done')
    all_chunks = preprocess_chunks(all_chunks)
    # Генерация эмбеддингов и сохранение их в Faiss
    embeddings = get_embeddings_st(all_chunks)
    print('2.1done')
    index = save_embeddings_to_faiss(np.array(embeddings))
    print('3done')
    # Сохранение изображений
    image_paths = save_images(all_images)
    print('4done')
    # Создание отображения чанков и изображений
    chunk_image_map = {i: image_paths[i] for i in range(len(all_images))}
    print('5done')
    # Статистика
    print(f"Обработано {len(pdf_files)} PDF файлов.")
    print(f"Чанков текста: {len(all_chunks)}.")
    print(f"Изображений: {len(all_images)}.")
    print("Эмбеддинги сохранены в Faiss, изображения в папке 'images'.")

    return all_chunks, chunk_image_map, index, embeddings


In [ ]:
!rm -r images

In [ ]:
pdf_folder_path = "pdfs"  # Укажите путь к папке с PDF файлами
all_chunks, chunk_image_map, index, embeddings = process_pdfs(pdf_folder_path)

In [ ]:
query = "калибровка значений с помощью весов"
similar_chunks_with_images = search_similar_chunks_with_images(query, all_chunks, chunk_image_map, index, embeddings, k=5)

print("Similar chunks with images:")
for i, (chunk, image) in enumerate(similar_chunks_with_images, start=1):
    print(f"Chunk {i}: {chunk}")
    print(f"Image: {image}")


In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="MTSAIR/Cotype-Nano", device="cuda")

messages = [

  {"role": "system", "content": "Ты — ИИ-помощник фермера тракториста. Тебе дано задание: необходимо сгенерировать емкий но полный ответ, отвечающий на поставленный вопрос, используя в качестве базы знаний поступающие тебе тексты."},

  {"role": "user", "content": "Расскажи мне про ИИ"},

]

res = pipe(messages, max_length=1024)

print(res[0]['generated_text'][-1]['content'])

In [ ]:
pipe2 = pipeline("text-generation", model="MTSAIR/Cotype-Nano", device="cuda")
messages2 = [
  {"role": "system", "content": "Ты — ИИ-помощник по правильности речи. Проверяй входящие фразы и исправляй грамматичексие, семантические и лексические ошибки, следи за структурой речи. Выведи только исправленную фразу (если она была неправильной, иначе - оставь как есть)"},
  {"role": "user", "content": "росскажи мене про искуственый интелект"},
]
res2 = pipe2(messages2, max_length=1024)
print(res2[0]['generated_text'][-1]['content'])

In [ ]:
!pip install 'accelerate>=0.26.0'

In [ ]:
import torch

# Clear all allocated tensors to free memory
torch.cuda.empty_cache()

# Optionally force garbage collection (not always necessary)
import gc
gc.collect()


In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    # Get the current GPU device
    device = torch.cuda.current_device()
    print(f"Device: {torch.cuda.get_device_name(device)}")
    
    # Get memory details
    total_memory = torch.cuda.get_device_properties(device).total_memory
    allocated_memory = torch.cuda.memory_allocated(device)
    cached_memory = torch.cuda.memory_reserved(device)
    free_memory = total_memory - allocated_memory - cached_memory
    
    print(f"Total Memory: {total_memory / 1024**3:.2f} GB")
    print(f"Allocated Memory: {allocated_memory / 1024**3:.2f} GB")
    print(f"Cached Memory: {cached_memory / 1024**3:.2f} GB")
    print(f"Free Memory: {free_memory / 1024**3:.2f} GB")
else:
    print("CUDA is not available.")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype="auto",
)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")



In [ ]:
def grammar(prompt):

    messages = [
        {"role": "system", "content": "Ты — ИИ-помощник по правильности речи. Проверяй входящие фразы и исправляй грамматичексие, семантические и лексические ошибки, следи за структурой речи. Выведи только исправленную фразу (если она была неправильной, иначе - оставь как есть)"},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response


In [ ]:
def helper_agent(similar_chunks, query):
    similar_chunks_text = "\n\n".join(similar_chunks)
    messages = [
        {"role": "system", "content": "Ты — ИИ-помощник фермера тракториста. Тебе дано задание: необходимо сгенерировать емкий но полный ответ, отвечающий на поставленный вопрос, используя в качестве базы знаний поступающие тебе тексты."},
        {"role": "user", "content": f"Вопрос: {query}\n\nТексты для анализа:\n{similar_chunks_text}"}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=4096
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return response


In [ ]:
def get_ans(question):
    # Preprocess the query
    query = grammar(question)
    
    # Search for similar chunks and associated images
    similar_chunks_with_images = search_similar_chunks_with_images(
        query=query,
        all_chunks=all_chunks,
        chunk_image_map=chunk_image_map,
        index=index,
        embeddings=embeddings,
        k=5  # Retrieve top-5 results
    )
    
    # Extract chunks (without images) for further processing
    similar_chunks = [chunk for chunk, _ in similar_chunks_with_images]
    similar_chunks_text = "\n\n".join(similar_chunks)
    
    # Pass similar chunks and query to helper_agent for generating response
    res = helper_agent(similar_chunks, query)

    # Debug the structure of res
    print(f"helper_agent output: {res}")
    
    # Since `res` is a string, directly return it
    return similar_chunks_with_images, res


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("test.csv")

In [ ]:
similar_chunks_list = []
answers_list = []

# Iterate over each row (assuming the question is in a column named 'question')
for question in df['user_input']:
    similar_chunks, answer = get_ans(question)
    similar_chunks_list.append(similar_chunks)  # Append the list directly
    answers_list.append(answer)

# Add results to new columns in the DataFrame
df2['contexts'] = similar_chunks_list  # Store the list directly
df2['response'] = answers_list

# Save the updated DataFrame back to a new CSV file
df2.to_csv("submission.csv", index=False)

In [ ]:
df['contexts'] = similar_chunks_list  # Store the list directly
df['response'] = answers_list

In [ ]:
print(df.columns)

In [ ]:
df = df.drop('user_input', axis=1)

In [ ]:
df.to_csv("submission.csv", index=False)

In [53]:
!pip install aiogram

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.4 MB/s eta 0:00:0000:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.27.1
    Uninstalling pydantic_core-2.27.1:
      Successfully uninstalled pydantic_core-2.27.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.10.3
    Uninstalling pydantic-2.10.3:
      Successfully uninstalled pydantic-2.10.3
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [ ]:
from aiogram import Bot, Dispatcher, Router
from aiogram.types import Message
from aiogram import F
import asyncio
import logging

# Включаем логирование
logging.basicConfig(level=logging.INFO)

# Укажите токен вашего бота
API_TOKEN = "7847179564:AAGWg7KkW23DxoPgPN5KtgnFyPNy-kXpnuQ"

# Инициализация бота и диспетчера
bot = Bot(token=API_TOKEN)
router = Router()
dp = Dispatcher()
dp.include_router(router)

@router.message(F.text)
async def handle_message(message: Message):
    # Преобразуем текст в нижний регистр
    _, res_text = get_ans(message.text)
    # Отправляем преобразованный текст пользователю
    await message.answer(res_text)

async def start():
    # Запуск бота
    await dp.start_polling(bot)

await start()


INFO:aiogram.dispatcher:Start polling
INFO:aiogram.dispatcher:Run polling for bot @vid_prof_agro_bot id=7847179564 - 'VID_ProfAgro_Bot'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Для проверки предоставленного текста и ответа на ваш вопрос о лакокрасочном покрытии для распределителей удобрений, давайте разберем ключевые моменты.

### Анализ текста:

1. **Технология лакирования**:
   - Применяется комбинированная технология, включающая катодное погружное лакирование (KTL) и метод порошкового напыления.
   - KTL используется для грунтования, обеспечивая защиту от коррозии.
   - Порошковое напыление обеспечивает высококачественный внешний вид и чрезвычайно плотный слой краски для защиты от механических воздействий.

2. **Преимущества**:
   - Высококачественное многослойное лакирование обеспечивает двойную защиту от механических воздействий и коррозии.
   - Процесс включает 14 этапов подготовки к лакированию, что гарантирует надежную защиту.

3. **Определение коэффициента калибровки**:
   - Для точного внесения удобрений используется система калибровки, которая включает в себя определение коэффициента калибровки через боковое устройство определе

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

helper_agent output: Если у вас возникли проблемы с трактором, важно учитывать несколько ключевых аспектов безопасности и эксплуатации:

1. **Фиксация трактора и агрегата**: Перед началом работы или устранением неисправностей всегда зафиксируйте трактор и агрегат от непреднамеренного пуска и откатывания. Это предотвратит аварийные ситуации, такие как захватывание или наматывание открытым первичным валом входного редуктора.

2. **Карданный вал и защитные приспособления**: Используйте только те карданные валы, которые указаны в списке допустимых. Убедитесь, что на карданном валу установлены все защитные приспособления и они работоспособны. Никогда не используйте карданный вал без защитных приспособлений или с поврежденными защитными приспособлениями.

3. **Безопасность при работе с двигателем**: Не поднимайтесь на погрузочную платформу, пока двигатель трактора работает, особенно если карданный вал подключен. Также никогда не открывайте и не снимайте защитные приспособления с движущихся ч

...

helper_agent output: Чтобы помочь вам в эксплуатации агрегата AMAZONE, важно следовать нескольким ключевым рекомендациям:

1. **Прочтение руководства по эксплуатации**: Перед началом работы обязательно ознакомьтесь с руководством по эксплуатации, которое включает в себя инструкции по использованию и обслуживанию агрегата. Это поможет избежать ошибок и повысит эффективность работы.

2. **Регулярное техническое обслуживание**: Проводите регулярное обслуживание агрегата, включая замену изношенных или поврежденных деталей. Это увеличит срок службы оборудования и обеспечит его надежную работу.

3. **Обновление руководств**: Наши руководства по эксплуатации регулярно обновляются на основе отзывов пользователей. Если у вас есть предложения или вопросы, пожалуйста, сообщите нам, чтобы мы могли улучшить руководства.

4. **Проверка гидравлических шлангопроводов**: При подключении гидравлических шлангопроводов убедитесь, что гидросистемы трактора и агрегата не находятся под давлением. Регулярно п

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:aiogram.event:Update id=716731309 is handled. Duration 83146 ms by bot id=7847179564
INFO:aiogram.event:Update id=716731310 is handled. Duration 13922 ms by bot id=7847179564
INFO:aiogram.event:Update id=716731308 is handled. Duration 83229 ms by bot id=7847179564


helper_agent output: Исходя из предоставленных текстов, информация о цвете трактора напрямую не связана с его функциональными характеристиками или техническими особенностями. Однако, в одном из текстов упоминается, что трактор имеет "синий" блок управления (подъем двойного действия) и "красный" напорный маслопровод системы LoadSensing. Это описание относится к цветовой маркировке элементов управления и систем трактора, а не к его внешнему цвету.

Что касается вашего вопроса: "Трактор был синим, а не красным", то в контексте этих текстов нет прямой информации о цвете самого трактора. Тексты описывают цвета различных элементов и систем трактора, таких как блок управления и маслопроводы, но не содержат данных о цвете самого корпуса трактора.

Если ваш вопрос касается конкретно цвета трактора, то, основываясь на предоставленной информации, мы можем сделать вывод, что цвет трактора не указан и не имеет отношения к его функциональным особенностям. Если же вам нужна информация о цвете трактор

Batches:   0%|          | 0/1 [00:00<?, ?it/s]